In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%reload_ext autoreload
%autoreload 2

from networks.models import load_model
from torch import cuda
from data.utils import SettingsTraining
from torch.utils.data import DataLoader
import torch
from main import init_data
import logging
from datetime import datetime
from typing import Dict
from math import inf
from torch.utils.data import DataLoader
from networks.unet import UNet
from utils.visualize_data import DataToVisualize, _aligned_colorbar

In [ ]:
def run(settings: SettingsTraining):
    dataset, dataloaders = init_data(settings)
    if settings.device is None:
        settings.device = "cuda" if cuda.is_available() else "cpu"
    # model choice
    in_channels = dataset.input_channels
    if settings.case in ["test", "finetune"]:
        model = load_model({"model_choice": settings.model_choice,
                           "in_channels": in_channels}, settings.path_to_model)
    model.to(settings.device)

    # load model (for application only)
    model = load_model({"model_choice": settings.model_choice, "in_channels": in_channels}, os.path.join(settings.path_to_model), "model", settings.device)
    model.to(settings.device)

    plot_sample(model, dataloaders["test"], settings.device, plot_name=settings.name_folder_destination + "/plot_test_sample", amount_plots=10,)


In [ ]:
def plot_sample(model: UNet, dataloader: DataLoader, device: str, amount_plots: int = inf, plot_name: str = "default"):

    logging.warning("Plotting...")
    error = []
    error_mean = []

    if amount_plots > len(dataloader.dataset):
        amount_plots = len(dataloader.dataset)

    current_id = 0
    norm = dataloader.dataset.dataset.norm
    for inputs, labels in dataloader:
        len_batch = inputs.shape[0]
        for datapoint_id in range(len_batch):
            # get data
            start_time = datetime.now()
            x = inputs[datapoint_id].to(device)
            x = torch.unsqueeze(x, 0)
            model.eval()
            y_out = model(x).to(device)

            # reverse transform for plotting real values
            y = labels[datapoint_id]
            x = norm.reverse(x.detach().cpu().squeeze(), "Inputs")
            y = norm.reverse(y.detach().cpu(),"Labels")[0]
            y_out = norm.reverse(y_out.detach().cpu()[0],"Labels")[0]
            print(f"Index {datapoint_id}")
            print(f"Time of inference: {(datetime.now() - start_time).total_seconds()}s")
            loc_max = y_out.argmax()
            print(f"Max temp: {y_out.max()} at {(loc_max%100, loc_max//100%1280, loc_max//100//1280%5)}")

            # calculate error
            error_current = y-y_out
            error.append(abs(error_current))
            error_mean.append(
                torch.mean(error_current).item())

            # plot temperature true, temperature out, error, physical variables
            temp_max = max(y.max(), y_out.max())
            temp_min = min(y.min(), y_out.min())
            info = dataloader.dataset.dataset.info
            extent_highs = (np.array(info["CellsSize"][:2]) * y.shape)
            dict_to_plot = {
                "t_true": DataToVisualize(y, "Temperature True [°C]",extent_highs, {"vmax": temp_max, "vmin": temp_min}),
                "t_out": DataToVisualize(y_out, "Temperature Out [°C]",extent_highs, {"vmax": temp_max, "vmin": temp_min}),
                "error": DataToVisualize(torch.abs(error_current), "Abs. Error [°C]",extent_highs),
            }
            physical_vars = info["Inputs"].keys()
            for physical_var in physical_vars:
                index = info["Inputs"][physical_var]["index"]
                dict_to_plot[physical_var] = DataToVisualize(
                    x[index], physical_var,extent_highs)

            name_pic = f"runs/{plot_name}_{current_id}"
            figsize_x = extent_highs[0]/extent_highs[1]*3
            _plot_datafields(dict_to_plot, name_pic=name_pic, figsize_x=figsize_x)

            logging.info(f"Resulting pictures are at runs/{plot_name}_*")

            # if (current_id > 0 and current_id % 6 == 0) or current_id >= amount_plots-1:
            #     plt.close("all")

            if current_id >= amount_plots-1:
                max_error = np.max(error[-1].cpu().numpy())
                logging.info("Maximum error: ", max_error)
                return error_mean, max_error
            current_id += 1

            break


In [ ]:
def _plot_datafields(data: Dict[str, DataToVisualize], name_pic: str, figsize_x: float = 38.4):
    _, axes = plt.subplots(3, 1, sharex=True,
                           figsize=(figsize_x, 3*(3)))

    for index, (name, datapoint) in enumerate(data.items()):
        plt.sca(axes[index])
        if name in ["t_true", "t_out", "error"]:  
            if name=="error": datapoint.contourargs["levels"] = [level - 10.6 for level in datapoint.contourargs["levels"]] 
            CS = plt.contour(torch.flip(datapoint.data, dims=[1]).T, **datapoint.contourargs)
            plt.clabel(CS, inline=1, fontsize=10)
            if index==1:
                path = CS.collections[0].get_paths()
                print(path[0])
        if index >= 2: break

In [ ]:
args = {
"datasets_path" : "datasets_prepared",
"device" : "cuda:3",
"epochs" : 30000,
"model_choice" : "unet",
"dataset_name" : "benchmark_testcases_4_power2trafo",
"case" : "test",
"path_to_model" : "current_unet_benchmark_dataset_2d_100datapoints_power2trafo",
}
args["name_folder_destination"] = f'current_{args["model_choice"]}_{args["dataset_name"]}'

settings = SettingsTraining(**args)
run(settings)

# Testcase 0

Index 0

Time of inference: 0.008557s

Max temp: 15.654577255249023 at (tensor(58), tensor(3), tensor(0))


In [ ]:
points = np.array([
       [134.46685127,  80.        ],
       [135.52941176,  79.57893545],
       [140.54901961,  77.5798159 ],
       [144.88336293,  74.66666667],
       [145.56862745,  74.225874  ],
       [150.58823529,  70.77884355],
       [152.24928155,  69.33333333],
       [155.60784314,  66.57058527],
       [158.43165394,  64.        ],
       [160.62745098,  61.26908855],
       [162.80156399,  58.66666667],
       [165.64705882,  54.28715017],
       [166.2923132 ,  53.33333333],
       [166.56847068,  52.87866797]])

In [ ]:
# get max and min of points[:,0] and points[:,1]
x_min = points[:,0].min()
x_max = points[:,0].max()
y_min = points[:,1].min()
y_max = points[:,1].max()
width = y_max - y_min
length = x_max - x_min
print(f"length = {length}, width = {width}")

# Testcase 1

Index 0

Time of inference: 0.006852s

Max temp: 15.43527603149414 at (tensor(7), tensor(4), tensor(0))


In [ ]:
points = np.array([
       [281.38030742,  55.63182652],
       [286.11764706,  55.52644632],
       [291.1372549 ,  55.52031584],
       [296.15686275,  55.4046529 ],
       [301.17647059,  55.37946446],
       [306.19607843,  55.22364652],
       [311.21568627,  55.13140275],
       [316.23529412,  54.88664707],
       [321.25490196,  54.67286408],
       [326.2745098 ,  54.31007399],
       [331.29411765,  53.99138788],
       [336.31372549,  53.53396875],
       [338.99696183,  53.33333333],
       [341.33333333,  53.20955146],
       [346.35294118,  52.84319239],
       [351.37254902,  52.56361732],
       [356.39215686,  52.17195537],
       [361.41176471,  51.90175951],
       [366.43137255,  51.49551784],
       [371.45098039,  51.2101942 ],
       [376.47058824,  50.74807537],
       [381.49019608,  50.42712243],
       [386.50980392,  49.86440562],
       [391.52941176,  49.45824259],
       [396.54901961,  48.73871536],
       [401.56862745,  48.24472776],
       [403.05091664,  48.        ],
       [406.58823529,  47.33927033],
       [411.60784314,  46.67041435],
       [416.62745098,  45.48936538],
       [421.64705882,  44.58671672],
       [426.66666667,  43.06190686],
       [428.2793511 ,  42.66666667],
       [431.68627451,  41.27629842],
       [436.70588235,  38.5265803 ],
       [439.19710618,  37.33333333],
       [436.70588235,  36.10452898],
       [431.68627451,  33.56627197],
       [428.38596994,  32.        ],
       [426.66666667,  31.60894993],
       [421.64705882,  30.65268344],
       [416.62745098,  29.65910026],
       [411.60784314,  28.94870102],
       [406.58823529,  28.11462557],
       [401.56862745,  27.52990391],
       [396.54901961,  26.84515095],
       [394.83189613,  26.66666667],
       [391.52941176,  26.27869881],
       [386.50980392,  25.5910138 ],
       [381.49019608,  25.10533695],
       [376.47058824,  24.51429093],
       [371.45098039,  24.15370185],
       [366.43137255,  23.65993509],
       [361.41176471,  23.37736343],
       [356.39215686,  22.96395905],
       [351.37254902,  22.75159646],
       [346.35294118,  22.37516509],
       [341.33333333,  22.19894316],
       [336.31372549,  21.8670593 ],
       [331.29411765,  21.7112973 ],
       [326.2745098 ,  21.41908674],
       [322.50428422,  21.33333333],
       [321.25490196,  21.29442643],
       [316.23529412,  20.96263488],
       [311.21568627,  20.86615476],
       [306.19607843,  20.61146726],
       [301.17647059,  20.54517063],
       [296.15686275,  20.33715119],
       [291.1372549 ,  20.26622688],
       [286.11764706,  20.10402143],
       [281.09803922,  19.97941795],
       [276.07843137,  19.85281399],
       [271.05882353,  19.7023032 ],
       [266.03921569,  19.62570311],
       [261.01960784,  19.44782289],
       [256.        ,  19.41257466],
       [250.98039216,  19.23206663],
       [245.96078431,  19.25460717],
       [240.94117647,  19.10232202],
       [235.92156863,  19.13258557],
       [230.90196078,  18.99775791],
       [225.88235294,  19.05294673],
       [220.8627451 ,  18.91955577],
       [215.84313725,  18.98679155],
       [210.82352941,  18.89552512],
       [205.80392157,  18.96462242],
       [200.78431373,  18.90916452],
       [195.76470588,  18.98513157],
       [190.74509804,  18.99567726],
       [185.7254902 ,  19.13788337],
       [180.70588235,  19.24362598],
       [175.68627451,  19.44720802],
       [170.66666667,  19.58951192],
       [165.64705882,  19.94402991],
       [160.62745098,  20.18211355],
       [155.60784314,  20.40817559],
       [150.58823529,  20.70603679],
       [145.56862745,  21.30785198],
       [140.54901961,  21.1308406 ],
       [137.82828431,  21.33333333],
       [135.52941176,  21.43035982],
       [130.50980392,  21.93580987],
       [125.49019608,  22.43154805],
       [120.47058824,  23.5105752 ],
       [115.45098039,  25.79592657],
       [114.6959021 ,  26.66666667],
       [111.24407729,  32.        ],
       [110.43137255,  35.71552898],
       [109.77925091,  37.33333333],
       [110.43137255,  39.35391335],
       [111.03442983,  42.66666667],
       [114.73160497,  48.        ],
       [115.45098039,  48.82196559],
       [120.47058824,  51.12955603],
       [125.49019608,  52.36105359],
       [130.50980392,  52.78019258],
       [135.52941176,  53.2212932 ],
       [140.54901961,  53.19700592],
       [142.97493363,  53.33333333],
       [145.56862745,  53.61081196],
       [150.58823529,  54.12481433],
       [155.60784314,  54.39152061],
       [160.62745098,  54.7027894 ],
       [165.64705882,  55.17727966],
       [170.66666667,  55.44603458],
       [175.68627451,  55.7157948 ],
       [180.70588235,  55.97185115],
       [185.7254902 ,  56.15480716],
       [190.74509804,  56.35910317],
       [195.76470588,  56.46027262],
       [200.78431373,  56.60849487],
       [205.80392157,  56.62484624],
       [210.82352941,  56.69165526],
       [215.84313725,  56.62677472],
       [220.8627451 ,  56.64037049],
       [225.88235294,  56.51735073],
       [230.90196078,  56.49554392],
       [235.92156863,  56.34240732],
       [240.94117647,  56.2849809 ],
       [245.96078431,  56.14534563],
       [250.98039216,  56.11456641],
       [256.        ,  55.95813784],
       [260.73788399,  55.93106997],
       ])

In [ ]:
# get max and min of points[:,0] and points[:,1]
x_min = points[:,0].min()
x_max = points[:,0].max()
y_min = points[:,1].min()
y_max = points[:,1].max()
width = y_max - y_min
length = x_max - x_min
print(f"length = {length}, width = {width}")

# Testcase 2

In [ ]:
points = np.array([
       [1004.20338157,   37.3899898 ],
       [1008.94117647,   37.50106723],
       [1013.96078431,   37.42512802],
       [1018.98039216,   37.54342588],
       [1024.        ,   37.45518456],
       [1029.01960784,   37.56376486],
       [1034.03921569,   37.47787406],
       [1039.05882353,   37.57768342],
       [1044.07843137,   37.49010993],
       [1049.09803922,   37.58332089],
       [1054.11764706,   37.49365016],
       [1059.1372549 ,   37.58512024],
       [1064.15686275,   37.49633777],
       [1069.17647059,   37.58142263],
       [1074.19607843,   37.48984523],
       [1079.21568627,   37.56864889],
       [1084.23529412,   37.4769749 ],
       [1089.25490196,   37.54883889],
       [1094.2745098 ,   37.45268005],
       [1099.29411765,   37.52197935],
       [1104.31372549,   37.43135246],
       [1109.33333333,   37.49968635],
       [1114.35294118,   37.41053067],
       [1119.37254902,   37.47821534],
       [1124.39215686,   37.38735208],
       [1129.41176471,   37.44867375],
       [1134.43137255,   37.36029584],
       [1139.45098039,   37.41884508],
       [1144.0273056 ,   37.33333333],
       [1139.45098039,   37.24880624],
       [1134.43137255,   37.30608612],
       [1129.41176471,   37.21897555],
       [1124.39215686,   37.27857619],
       [1119.37254902,   37.18941546],
       [1114.35294118,   37.25493392],
       [1109.33333333,   37.16785606],
       [1104.31372549,   37.23366516],
       [1099.29411765,   37.14544861],
       [1094.2745098 ,   37.21176606],
       [1089.25490196,   37.11875779],
       [1084.23529412,   37.18710572],
       [1079.21568627,   37.09900584],
       [1074.19607843,   37.1740869 ],
       [1069.17647059,   37.08597698],
       [1064.15686275,   37.16757324],
       [1059.1372549 ,   37.08230633],
       [1054.11764706,   37.17074374],
       [1049.09803922,   37.08454476],
       [1044.07843137,   37.17484619],
       [1039.05882353,   37.09077729],
       [1034.03921569,   37.18793337],
       [1029.01960784,   37.10540307],
       [1024.        ,   37.21130782],
       [1018.98039216,   37.12607779],
       [1013.96078431,   37.24162081],
       [1008.94117647,   37.1680907 ],
       [1003.92156863,   37.28339526],
       [ 998.90196078,   37.1934141 ],
       [ 993.88235294,   37.29631596],
       [ 988.8627451 ,   37.18960424],
       [ 983.84313725,   37.28207999],
       [ 978.82352941,   37.17663282],
       [ 973.80392157,   37.25839398],
       [ 968.78431373,   37.1493179 ],
       [ 963.76470588,   37.22255218],
       [ 958.74509804,   37.11616292],
       [ 953.7254902 ,   37.17740678],
       [ 948.70588235,   37.06768848],
       [ 943.68627451,   37.12428419],
       [ 938.66666667,   37.02647839],
       [ 933.64705882,   37.09542786],
       [ 928.62745098,   37.02126342],
       [ 923.60784314,   37.10374301],
       [ 918.58823529,   37.03874793],
       [ 913.56862745,   37.12020108],
       [ 908.54901961,   37.05993958],
       [ 903.52941176,   37.13796   ],
       [ 898.50980392,   37.08520583],
       [ 893.49019608,   37.15801364],
       [ 888.47058824,   37.1041048 ],
       [ 883.45098039,   37.1691381 ],
       [ 878.43137255,   37.1155396 ],
       [ 873.41176471,   37.17542305],
       [ 868.39215686,   37.11934599],
       [ 863.37254902,   37.17070631],
       [ 858.35294118,   37.11093113],
       [ 853.33333333,   37.15257568],
       [ 848.31372549,   37.08453542],
       [ 843.29411765,   37.11568071],
       [ 838.2745098 ,   37.04590393],
       [ 833.25490196,   37.06946354],
       [ 828.23529412,   37.00793402],
       [ 823.21568627,   37.01829906],
       [ 818.19607843,   36.95727804],
       [ 813.17647059,   36.96124567],
       [ 808.15686275,   36.90489421],
       [ 803.1372549 ,   36.90192367],
       [ 798.11764706,   36.84807238],
       [ 793.09803922,   36.84302129],
       [ 788.07843137,   36.79265881],
       [ 783.05882353,   36.78496256],
       [ 778.03921569,   36.7453619 ],
       [ 773.01960784,   36.73863884],
       [ 768.        ,   36.70494927],
       [ 762.98039216,   36.70090214],
       [ 757.96078431,   36.67044863],
       [ 752.94117647,   36.67028415],
       [ 747.92156863,   36.64455489],
       [ 742.90196078,   36.66863961],
       [ 737.88235294,   36.6348388 ],
       [ 732.8627451 ,   36.65138234],
       [ 727.84313725,   36.59990148],
       [ 722.82352941,   36.61332261],
       [ 717.80392157,   36.55058266],
       [ 712.78431373,   36.56977282],
       [ 707.76470588,   36.51031495],
       [ 702.74509804,   36.54333705],
       [ 697.7254902 ,   36.48856772],
       [ 692.70588235,   36.52740599],
       [ 687.68627451,   36.48563005],
       [ 682.66666667,   36.52383715],
       [ 677.64705882,   36.49309262],
       [ 672.62745098,   36.51671785],
       [ 667.60784314,   36.48690433],
       [ 662.58823529,   36.50160721],
       [ 657.56862745,   36.49141386],
       [ 652.54901961,   36.49882189],
       [ 647.52941176,   36.50182314],
       [ 642.50980392,   36.50950069],
       [ 637.49019608,   36.51524093],
       [ 632.47058824,   36.52524423],
       [ 627.45098039,   36.52361619],
       [ 622.43137255,   36.53035815],
       [ 617.41176471,   36.52440476],
       [ 612.39215686,   36.5252006 ],
       [ 607.37254902,   36.52416233],
       [ 602.35294118,   36.51435216],
       [ 597.33333333,   36.52517901],
       [ 592.31372549,   36.49530699],
       [ 587.29411765,   36.510727  ],
       [ 582.2745098 ,   36.46035368],
       [ 577.25490196,   36.47192522],
       [ 572.23529412,   36.42121733],
       [ 567.21568627,   36.42509225],
       [ 562.19607843,   36.38231124],
       [ 557.17647059,   36.39438618],
       [ 552.15686275,   36.36966259],
       [ 547.1372549 ,   36.38731621],
       [ 542.11764706,   36.36392247],
       [ 537.09803922,   36.36811   ],
       [ 532.07843137,   36.34320157],
       [ 527.05882353,   36.33674818],
       [ 522.03921569,   36.31579784],
       [ 517.01960784,   36.29762017],
       [ 512.        ,   36.27443603],
       [ 506.98039216,   36.2497636 ],
       [ 501.96078431,   36.22682585],
       [ 496.94117647,   36.20808936],
       [ 491.92156863,   36.17895814],
       [ 486.90196078,   36.14486144],
       [ 481.88235294,   36.10193985],
       [ 476.8627451 ,   36.05841358],
       [ 471.84313725,   36.01358589],
       [ 466.82352941,   35.95326453],
       [ 461.80392157,   35.8952663 ],
       [ 456.78431373,   35.83087473],
       [ 451.76470588,   35.7567107 ],
       [ 446.74509804,   35.69201818],
       [ 441.7254902 ,   35.62568919],
       [ 436.70588235,   35.56725701],
       [ 431.68627451,   35.52695749],
       [ 426.66666667,   35.4643131 ],
       [ 421.64705882,   35.43038115],
       [ 416.62745098,   35.37051679],
       [ 411.60784314,   35.33928775],
       [ 406.58823529,   35.27581668],
       [ 401.56862745,   35.25739753],
       [ 396.54901961,   35.21029248],
       [ 391.52941176,   35.20085608],
       [ 386.50980392,   35.16055848],
       [ 381.49019608,   35.14689533],
       [ 376.47058824,   35.10437758],
       [ 371.45098039,   35.0861718 ],
       [ 366.43137255,   35.04170954],
       [ 361.41176471,   35.03008124],
       [ 356.39215686,   34.9930504 ],
       [ 351.37254902,   34.99098546],
       [ 346.35294118,   34.96477185],
       [ 341.33333333,   34.97199392],
       [ 336.31372549,   34.95158796],
       [ 331.29411765,   34.96670481],
       [ 326.2745098 ,   34.95065369],
       [ 321.25490196,   34.97590672],
       [ 316.23529412,   34.96328639],
       [ 311.21568627,   35.00205877],
       [ 306.19607843,   34.98833726],
       [ 301.17647059,   35.03501147],
       [ 296.15686275,   35.00649447],
       [ 291.1372549 ,   35.05305239],
       [ 286.11764706,   35.00833998],
       [ 281.09803922,   35.052264  ],
       [ 276.07843137,   34.99823383],
       [ 271.05882353,   35.04433791],
       [ 266.03921569,   34.98303839],
       [ 261.01960784,   35.0284564 ],
       [ 256.        ,   34.95784145],
       [ 250.98039216,   35.00625813],
       [ 245.96078431,   34.92351782],
       [ 240.94117647,   34.96684845],
       [ 235.92156863,   34.86975247],
       [ 230.90196078,   34.90470875],
       [ 225.88235294,   34.79711701],
       [ 220.8627451 ,   34.83115159],
       [ 215.84313725,   34.71461962],
       [ 210.82352941,   34.76479444],
       [ 205.80392157,   34.65097118],
       [ 200.78431373,   34.71987905],
       [ 195.76470588,   34.6020284 ],
       [ 190.74509804,   34.67470716],
       [ 185.7254902 ,   34.55924828],
       [ 180.70588235,   34.6505465 ],
       [ 175.68627451,   34.53782356],
       [ 170.66666667,   34.62856115],
       [ 165.64705882,   34.49400586],
       [ 160.62745098,   34.56250332],
       [ 155.60784314,   34.43980289],
       [ 150.58823529,   34.528656  ],
       [ 145.56862745,   34.36209196],
       [ 140.54901961,   34.4024911 ],
       [ 135.52941176,   34.44120876],
       [ 130.50980392,   34.40561133],
       [ 125.49019608,   34.38859446],
       [ 120.47058824,   34.589655  ],
       [ 115.45098039,   34.70971517],
       [ 113.20497447,   37.33333333],
       [ 115.45098039,   40.0301356 ],
       [ 120.47058824,   40.15614702],
       [ 125.49019608,   40.38417179],
       [ 130.50980392,   40.29521643],
       [ 135.52941176,   40.30173703],
       [ 140.54901961,   40.33379988],
       [ 145.56862745,   40.31320672],
       [ 150.58823529,   40.20231333],
       [ 155.60784314,   40.24693893],
       [ 160.62745098,   40.16000492],
       [ 165.64705882,   40.1812555 ],
       [ 170.66666667,   40.07988559],
       [ 175.68627451,   40.1297073 ],
       [ 180.70588235,   40.05908871],
       [ 185.7254902 ,   40.10447136],
       [ 190.74509804,   40.0359799 ],
       [ 195.76470588,   40.06425058],
       [ 200.78431373,   39.98793454],
       [ 205.80392157,   40.01343589],
       [ 210.82352941,   39.94269913],
       [ 215.84313725,   39.95429245],
       [ 220.8627451 ,   39.88034296],
       [ 225.88235294,   39.8836315 ],
       [ 230.90196078,   39.8143053 ],
       [ 235.92156863,   39.82613167],
       [ 240.94117647,   39.76317165],
       [ 245.96078431,   39.78583557],
       [ 250.98039216,   39.73408484],
       [ 256.        ,   39.76435623],
       [ 261.01960784,   39.72144792],
       [ 266.03921569,   39.75087607],
       [ 271.05882353,   39.71710042],
       [ 276.07843137,   39.74471461],
       [ 281.09803922,   39.72065514],
       [ 286.11764706,   39.73833681],
       [ 291.1372549 ,   39.72559667],
       [ 296.15686275,   39.73754925],
       [ 301.17647059,   39.74147971],
       [ 306.19607843,   39.75035419],
       [ 311.21568627,   39.77188535],
       [ 316.23529412,   39.76780878],
       [ 321.25490196,   39.79295492],
       [ 326.2745098 ,   39.76972166],
       [ 331.29411765,   39.79384243],
       [ 336.31372549,   39.76103599],
       [ 341.33333333,   39.77288134],
       [ 346.35294118,   39.73631567],
       [ 351.37254902,   39.73864909],
       [ 356.39215686,   39.69537237],
       [ 361.41176471,   39.68750983],
       [ 366.43137255,   39.63845822],
       [ 371.45098039,   39.62531897],
       [ 376.47058824,   39.5739893 ],
       [ 381.49019608,   39.56555213],
       [ 386.50980392,   39.51286074],
       [ 391.52941176,   39.51461663],
       [ 396.54901961,   39.46062992],
       [ 401.56862745,   39.46049957],
       [ 406.58823529,   39.391765  ],
       [ 411.60784314,   39.37995673],
       [ 416.62745098,   39.29835399],
       [ 421.64705882,   39.28659645],
       [ 426.66666667,   39.20334053],
       [ 431.68627451,   39.18580632],
       [ 436.70588235,   39.10343108],
       [ 441.7254902 ,   39.08234309],
       [ 446.74509804,   38.98511185],
       [ 451.76470588,   38.95033602],
       [ 456.78431373,   38.85166586],
       [ 461.80392157,   38.81300073],
       [ 466.82352941,   38.73142079],
       [ 471.84313725,   38.69624247],
       [ 476.8627451 ,   38.62815252],
       [ 481.88235294,   38.60594647],
       [ 486.90196078,   38.5387844 ],
       [ 491.92156863,   38.52624191],
       [ 496.94117647,   38.47228287],
       [ 501.96078431,   38.47591813],
       [ 506.98039216,   38.42750665],
       [ 512.        ,   38.42330573],
       [ 517.01960784,   38.37592362],
       [ 522.03921569,   38.37502312],
       [ 527.05882353,   38.33120057],
       [ 532.07843137,   38.34126811],
       [ 537.09803922,   38.29569276],
       [ 542.11764706,   38.31466399],
       [ 547.1372549 ,   38.27360507],
       [ 552.15686275,   38.30322597],
       [ 557.17647059,   38.26419557],
       [ 562.19607843,   38.28875818],
       [ 567.21568627,   38.23327546],
       [ 572.23529412,   38.24732032],
       [ 577.25490196,   38.18617457],
       [ 582.2745098 ,   38.20602001],
       [ 587.29411765,   38.14600958],
       [ 592.31372549,   38.17051851],
       [ 597.33333333,   38.13133083],
       [ 602.35294118,   38.1511772 ],
       [ 607.37254902,   38.13251078],
       [ 612.39215686,   38.1399006 ],
       [ 617.41176471,   38.13227732],
       [ 622.43137255,   38.13317567],
       [ 627.45098039,   38.13322061],
       [ 632.47058824,   38.13665849],
       [ 637.49019608,   38.14224694],
       [ 642.50980392,   38.1516602 ],
       [ 647.52941176,   38.15606999],
       [ 652.54901961,   38.16027149],
       [ 657.56862745,   38.16608641],
       [ 662.58823529,   38.15495985],
       [ 667.60784314,   38.16764925],
       [ 672.62745098,   38.13584086],
       [ 677.64705882,   38.15887569],
       [ 682.66666667,   38.12506944],
       [ 687.68627451,   38.16496113],
       [ 692.70588235,   38.11988571],
       [ 697.7254902 ,   38.16271502],
       [ 702.74509804,   38.10467712],
       [ 707.76470588,   38.14204324],
       [ 712.78431373,   38.07670205],
       [ 717.80392157,   38.10238359],
       [ 722.82352941,   38.03240603],
       [ 727.84313725,   38.05368523],
       [ 732.8627451 ,   37.99423821],
       [ 737.88235294,   38.02064526],
       [ 742.90196078,   37.97877956],
       [ 747.92156863,   38.01270198],
       [ 752.94117647,   37.97726642],
       [ 757.96078431,   37.98789181],
       [ 762.98039216,   37.9474226 ],
       [ 768.        ,   37.95413102],
       [ 773.01960784,   37.91184443],
       [ 778.03921569,   37.91506671],
       [ 783.05882353,   37.86772574],
       [ 788.07843137,   37.86986948],
       [ 793.09803922,   37.81211526],
       [ 798.11764706,   37.8163241 ],
       [ 803.1372549 ,   37.75601651],
       [ 808.15686275,   37.76201566],
       [ 813.17647059,   37.69912548],
       [ 818.19607843,   37.71148276],
       [ 823.21568627,   37.64394352],
       [ 828.23529412,   37.66164448],
       [ 833.25490196,   37.5941324 ],
       [ 838.2745098 ,   37.62397375],
       [ 843.29411765,   37.54926014],
       [ 848.31372549,   37.58538858],
       [ 853.33333333,   37.5131598 ],
       [ 858.35294118,   37.55911052],
       [ 863.37254902,   37.49542389],
       [ 868.39215686,   37.55104422],
       [ 873.41176471,   37.49116464],
       [ 878.43137255,   37.55547475],
       [ 883.45098039,   37.49794628],
       [ 888.47058824,   37.56772135],
       [ 893.49019608,   37.50963873],
       [ 898.50980392,   37.58766852],
       [ 903.52941176,   37.53037834],
       [ 908.54901961,   37.61445732],
       [ 913.56862745,   37.54918666],
       [ 918.58823529,   37.63714645],
       [ 923.60784314,   37.56639822],
       [ 928.62745098,   37.65554668],
       [ 933.64705882,   37.57501238],
       [ 938.66666667,   37.64992314],
       [ 943.68627451,   37.54532665],
       [ 948.70588235,   37.60662031],
       [ 953.7254902 ,   37.49099897],
       [ 958.74509804,   37.55622655],
       [ 963.76470588,   37.44506141],
       [ 968.78431373,   37.52163318],
       [ 973.80392157,   37.40872311],
       [ 978.82352941,   37.49318828],
       [ 983.5614241 ,   37.39089563],
])


In [ ]:
# get max and min of points[:,0] and points[:,1]
x_min = points[:,0].min()
x_max = points[:,0].max()
y_min = points[:,1].min()
y_max = points[:,1].max()
width = y_max - y_min
length = x_max - x_min
print(f"length = {length}, width = {width}")

# Testcase 3

In [ ]:
# Index 0
# Time of inference: 0.00679s
# Max temp: 11.10051441192627 at (tensor(71), tensor(4), tensor(0))
# Path(array([[1280.,   16.]]), None)

In [ ]:
widht = 0
length = 0